In [1]:
import pyemma
import numpy as np
import pyemma.coordinates as coor
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import os
from matplotlib.pyplot import cm
from collections import OrderedDict
import mdtraj as md
import itertools
import time
import indices
from indices.base import BaseComparisons as bc
from indices.faith import Faith as Fai
import scipy.cluster.hierarchy as shc
from matplotlib import gridspec
from matplotlib import pyplot
import mdtraj
import seaborn as sns

In [ ]:
Threshold = 0.6
traj_prefix = '/orange/alberto.perezant/liweichang/msm/MultipleComparisons/datasets'
pdb_loc = traj_prefix+'/DESRES-Trajectory_A3D-0-c-alpha/A3D-0-c-alpha/test_pdb/A3D.pdb'
traj = md.load_dcd(traj_prefix+'/DESRES-Trajectory_A3D-0-c-alpha/A3D-0-c-alpha/A3D-0-c-alpha-000.dcd',top=pdb_loc)
pdb_top = md.load_pdb(pdb_loc)
topfile=traj.top
feat = coor.featurizer(topfile)
residues = np.arange(0,pdb_top.n_residues)
pairs = []                                                                                 
for i,r1 in enumerate(residues):
    for r2 in residues[i+1::2]:
        pairs.append([r1,r2])
pairs = np.array(pairs)
feature=feat.add_residue_mindist(pairs, scheme='closest-heavy',threshold=Threshold,periodic=False)
trajfile = []
a=open(traj_prefix+'/DESRES-Trajectory_A3D_result/A3D_traj.dat','r')
a=a.readlines()            
for i in range(len(a)):    
    trajfile.append(a[i][:-1])
for index,i in enumerate(trajfile):
    if index == 0:
        inp = pyemma.coordinates.load(trajfile[0], features=feat)
    else:
        temp_inp = pyemma.coordinates.load(i, features=feat)
        inp = np.vstack((inp,temp_inp))

In [20]:
traj_prefix = '/orange/alberto.perezant/liweichang/msm/MultipleComparisons/datasets'
pdb_loc = traj_prefix+'/DESRES-Trajectory_A3D-0-c-alpha/A3D-0-c-alpha/test_pdb/A3D.pdb'
sample_index=np.random.randint(0,3000000,5000)
u,c=np.unique(sample_index,return_counts=True)
sample_index = u[:4000]
R = np.concatenate(np.load(traj_prefix+'/DESRES-Trajectory_A3D_result/feat_A3D.npz',allow_pickle=True)['arr_0'])[sample_index]

In [25]:
a=np.array(range(10))
np.delete(a,[1,2])
R = np.concatenate(np.load(traj_prefix+'/DESRES-Trajectory_A3D_result/feat_A3D.npz',allow_pickle=True)['arr_0'])
R.shape

(3535447, 1332)

In [49]:
# R = np.concatenate(np.load(traj_prefix+'/mdm2_ATSP7041/feat_ATSP.npz',allow_pickle=True)['arr_0'])
# R.shape
a=[(i)*19998+d for d in range(9999,19998) for i in range(30)]
len(a)

299970

In [31]:
all_start = time.time()
all_input = list(itertools.combinations(inp, 2))
batch_size=1000000
inp_sliced=[all_input[i*batch_size:(i+1)*batch_size] for i in range(int(len(all_input)/batch_size))]
if int(len(inp_sliced)) < len(all_input)/batch_size:
    inp_sliced.append(all_input[len(inp_sliced)*batch_size:])
for i in range(len(inp_sliced)):
    temp_start = time.time()
    temp_c = np.zeros((int(len(inp_sliced[i])),3))
    temp_input = np.array(inp_sliced[i])
    product = temp_input.reshape(-1,2,temp_input.shape[-1]).sum(1)
    for row in range(3):
        temp_c[:,row] = np.sum(product==row,axis=1)
    if i == 0:
        all_c = temp_c
    else:
        all_c = np.concatenate((all_c,temp_c),axis=0)
    print('all_c shape =', all_c.shape)
    temp_end = time.time()
    temp_time = temp_end - temp_start
    print('step {} time ='.format(i),round(temp_time,2),'s')


###calculate similarity
simi = all_c[:,0]+all_c[:,2]
###try Faith
Faith=False
if Faith:
    faith_simi = 0.5*all_c[:,0]+all_c[:,2]
    denominate = all_c[:,0]+all_c[:,1]+all_c[:,2]
    w_p = faith_simi/denominate
    simi_matrix = np.zeros((len(inp),len(inp)))
    indices = np.triu_indices(len(inp),k=1)
    indices = (indices[1],indices[0])
    simi_matrix[indices] = w_p
else:
    simi_matrix = np.zeros((len(inp),len(inp)))
    indices = np.triu_indices(len(inp),k=1)
    indices = (indices[1],indices[0])
    simi_matrix[indices] = simi
print('final matrix shape = ', simi_matrix.shape)
all_end = time.time()
all_time = all_end - all_start
print('total time =', round(all_time,2),'s')

NameError: name 'inp' is not defined

In [9]:
inp

array([[1., 1., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.]])

In [10]:
import pandas  as pd
all_start=time.time()
inp_copy=inp
df = pd.DataFrame(simi_matrix,columns=pd.MultiIndex.from_tuples([('{}'.format(i),'{}'.format(i)) for i in range(1,len(inp_copy)+1)],names=['cluster', 'frame']))
dic={}
hie_tree=[]
dentrom=[]
time_step=[]
while df.shape[0] > 2:
    ###update df
    frame_column=[i[1] for i in df.columns.to_list()]
    hie_tree.append(frame_column)
    new_max=np.argmax(df, axis=None)     ###2.5s

    del_index = np.unravel_index(new_max, df.shape)
    max_value = df.to_numpy()[del_index[0]][del_index[1]] ###0.1s

    ###get temp inp
    delete_row=[int(i) for i in df.columns[del_index[0]][1].split(',') ]
    delete_cluster_row=[int(i) for i in df.columns[del_index[0]][0].split(',')]
    delete_column=[int(i) for i in df.columns[del_index[1]][1].split(',') ]
    delete_cluster_column=[int(i) for i in df.columns[del_index[1]][0].split(',')]
    dentrom.append([*delete_cluster_column,*delete_cluster_row])

    delete_all=delete_column+delete_row
    insert_index='{}'.format(delete_all)[1:-1]   ###0.1s
    insert_cluster_index='{}'.format(2*len(inp) - df.shape[0]+1)
    df.drop(columns=[('{}'.format(delete_cluster_column)[1:-1],'{}'.format(delete_column)[1:-1]),('{}'.format(delete_cluster_row)[1:-1],'{}'.format(delete_row)[1:-1])],axis=1,inplace=True)
    df.drop(index=[*del_index],axis=0,inplace=True)
    df.reset_index(drop=True,inplace=True)
    temp_w_sim=[]            ###1s

    ###get temp inp
   # print(df.columns)
    frame_column=[i[1] for i in df.columns.to_list() ]
    columns=[[int(d) for d in [*i.split(',')]] for i in frame_column]
    temp_inp=[[inp_copy[d-1] for d in m] for m in columns]    ###0.1s

    ###perform comparison
#     if Faith:
#         for i in temp_inp:
#             compare=Fai(np.concatenate(([inp_copy[d-1] for d in delete_all],i),axis=0))
#             temp_w_sim.append(compare.Fai_1sim_wdis)
#     else:
    for i in temp_inp:
       # compare=bc(np.concatenate(([inp_copy[d-1] for d in delete_all],i),axis=0))
        compare=bc(np.concatenate((inp_copy[np.array(delete_all)-1],i),axis=0))
        temp_w_sim.append(compare.total_w_sim)         ###1s

#     temp_start = time.time()
  #  inp_copy = temp_inp
    temp_w_sim=[0]+temp_w_sim
    df.loc[-1] = [0]*df.shape[1] # adding a row
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index
   # print(df.shape)
    if df.shape[0] % 50 == 0:
        all_end = time.time()
        all_time = all_end - all_start
        print('merge {} clusters time ='.format(df.shape[0]), round(all_time,2),'s')
        time_step.append(round(all_time,4))
    df.insert(loc=0, column=(insert_cluster_index,insert_index), value=temp_w_sim)

    dic[insert_index]=max_value
temp_end = time.time() 
temp_time = temp_end - all_start          
print('merge {} clusters time ='.format(df.shape[0]), round(temp_time,2),'s')
time_step.append(round(temp_time,4))
last_two = [int(i[0]) for i in df.columns.to_list()]
dentrogram = np.vstack(dentrom)-np.ones((1,2))
values = np.max(simi_matrix)-np.array([*dic.values()],ndmin=2)+2
num_frames = np.array([len(i) for i in [[int(d) for d in [*k.split(',')]] for k in list(dic.keys())]],ndmin=2)
tree = np.hstack((dentrogram,values.T,num_frames.T))
tree = np.vstack((tree, [[last_two[0]-1,last_two[1]-1,np.max(simi_matrix)-df.to_numpy()[1][0],len(inp)]]))

merge 11750 clusters time = 713.52 s
merge 11700 clusters time = 1722.45 s
merge 11650 clusters time = 2775.53 s
merge 11600 clusters time = 3805.43 s
merge 11550 clusters time = 4810.42 s
merge 11500 clusters time = 5796.95 s
merge 11450 clusters time = 6753.65 s
merge 11400 clusters time = 7825.93 s
merge 11350 clusters time = 8898.39 s
merge 11300 clusters time = 9994.89 s
merge 11250 clusters time = 10970.53 s
merge 11200 clusters time = 11915.01 s
merge 11150 clusters time = 12835.35 s
merge 11100 clusters time = 13745.23 s
merge 11050 clusters time = 14649.55 s
merge 11000 clusters time = 15559.2 s
merge 10950 clusters time = 16455.26 s
merge 10900 clusters time = 17523.75 s
merge 10850 clusters time = 18564.47 s
merge 10800 clusters time = 19614.67 s
merge 10750 clusters time = 20596.36 s
merge 10700 clusters time = 21547.97 s
merge 10650 clusters time = 22485.54 s
merge 10600 clusters time = 23312.35 s
merge 10550 clusters time = 24091.62 s


KeyboardInterrupt: 